In [1]:
import pandas as pd
from sqlalchemy import create_engine, URL
from tqdm import tqdm

In [10]:
url_object = URL.create(
    drivername='postgresql+psycopg2',
    username='tie',
    password='TIE%2023!tuhh',
    host='134.28.58.100',
    port=25432,
    database='Patstat',
)

In [16]:
engine = create_engine(url_object)
# Check if connection is successful
engine.connect()

DETAIL:  The database was created using collation version 2.31, but the operating system provides version 2.36.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE "Patstat" REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.


In [23]:
query_cpc = """
    SELECT appln_id, cpc_class_symbol
    FROM public.tls224_appln_cpc
    WHERE cpc_class_symbol LIKE 'Y02%%';
"""

In [24]:
df_cpc_epo = pd.read_sql_query(query_cpc, engine)

In [25]:
# Group by appln_id and aggregate cpc_class_symbol
df_cpc_epo_list = df_cpc_epo.groupby('appln_id')['cpc_class_symbol'].apply(list).reset_index()

In [31]:
# Cast appln_id to string
df_cpc_epo_list['appln_id'] = df_cpc_epo_list['appln_id'].astype(str)

In [32]:
# Delete everything after "." in appln_id
df_cpc_epo_list['appln_id'] = df_cpc_epo_list['appln_id'].str.split('.').str[0]

In [65]:
# Insert temporary table into database
df_cpc_epo_list.to_sql('temp_cpc_epo', engine, if_exists='replace', index=False)

543

In [76]:
# SQL query to get appln_auth and granted from tls201_appln (casting appln_id to double precision)
query_appln = """
    SELECT 
        tls201_appln.appln_id, 
        tls201_appln.appln_auth, 
        tls201_appln.granted
    FROM 
        tls201_appln
    JOIN 
        temp_cpc_epo 
    ON 
        tls201_appln.appln_id = CAST(temp_cpc_epo.appln_id AS DOUBLE PRECISION);
"""

In [77]:
# Run query for each appln_id in epo_appln_id_list
df_appln_cpc_epo = pd.read_sql(query_appln, engine)

In [82]:
# Drop all rows where appln_auth not equal to EP
df_appln_cpc_ep = df_appln_cpc_epo[df_appln_cpc_epo['appln_auth'] == 'EP']

In [85]:
# Drop all rows where granted not equal to Y
df_appln_cpc_ep_granted = df_appln_cpc_ep[df_appln_cpc_ep['granted'] == 'Y']

In [96]:
# Throw away everything after dot in appln_id
df_appln_cpc_ep_granted['appln_id'] = df_appln_cpc_ep_granted['appln_id'].str.split('.').str[0]
df_appln_cpc_epo['appln_id'] = df_appln_cpc_epo['appln_id'].str.split('.').str[0]

# Cast appln_id to string
df_appln_cpc_ep_granted['appln_id'] = df_appln_cpc_ep_granted['appln_id'].astype(str)
df_appln_cpc_epo['appln_id'] = df_appln_cpc_epo['appln_id'].astype(str)

/tmp/ipykernel_1823398/1341935432.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_appln_cpc_ep_granted['appln_id'] = df_appln_cpc_ep_granted['appln_id'].str.split('.').str[0]
/tmp/ipykernel_1823398/1341935432.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_appln_cpc_ep_granted['appln_id'] = df_appln_cpc_ep_granted['appln_id'].astype(str)


In [98]:
# Merge df_cpc_ep and df_appln_cpc_ep_granted on appln_id to get cpc_class_symbol for granted patents
df_appln_cpc_ep_granted = df_appln_cpc_ep_granted.merge(df_cpc_epo_list, on='appln_id', how='left')
df_appln_cpc_epo = df_appln_cpc_epo.merge(df_cpc_epo_list, on='appln_id', how='left')

In [102]:
# Save both dataframes to csv
df_appln_cpc_ep_granted.to_csv('/mnt/hdd01/PATSTAT Working Directory/cleantech_ep_granted.csv', index=False)
df_appln_cpc_epo.to_csv('/mnt/hdd01/PATSTAT Working Directory/cleantech_patstat_all.csv', index=False)